In [1]:
# Model
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig, LlamaForCausalLM
from langchain_huggingface import HuggingFacePipeline
import torch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
# Vector stores
import fitz  # PyMuPDF
import pdfplumber
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
#from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader
#from langchain.text_splitter import RecursiveCharacterTextSplitter, KonlpyTextSplitter, MarkdownHeaderTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter
#from langchain_community.retrievers import BM25Retriever, KNNRetriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
#from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader ,UnstructuredPDFLoader
from langchain_community.retrievers import BM25Retriever, KNNRetriever
from langchain.retrievers import EnsembleRetriever
#from langchain_teddynote.retrievers import KiwiBM25Retriever, OktBM25Retriever
from langchain_teddynote.retrievers import OktBM25Retriever
from langchain.docstore.document import Document
#from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import unicodedata
import pymupdf4llm
#import time
#import re
#from konlpy.tag import Okt
#from pdf2image import convert_from_path
import pytesseract
from konlpy.tag import Kkma
# etc
#import os
import pandas as pd
from tqdm import tqdm
import unicodedata
import logging
#from PyPDF2 import PdfReader
#import json
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # GPU 사용 가능 여부 및 MPS 지원 여부 확인
print(device)

c:\ProgramData\anaconda3\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
# intfloat/multilingual-e5-small
# jhgan/ko-sroberta-multitask

def get_embedding():
    
    embeddings = HuggingFaceEmbeddings(
        model_name='jhgan/ko-sroberta-multitask',
        model_kwargs={'device': device},
        encode_kwargs={'normalize_embeddings': True})
    return embeddings
def normalize_string(s):
    try:
        normalized = unicodedata.normalize('NFC', s)
        return normalized.encode('utf-8', errors='replace').decode('utf-8')
    except Exception:
        return s
def clean_text(text):
    text = text.replace("�", " ").replace("", " ")  # 잘못된 인코딩 문자 제거
    return text

def format_docs(docs):
    context = ""
    i = 1
    for doc in docs:
        #context += f"Document: {i}" +"Source:"+ doc.metadata['source'] +'\n'
        context += doc.page_content
        context += '\n---\n'
        i += 1
    return context


In [3]:
def process_pdf(pdf_path): 
    md_text = pymupdf4llm.to_markdown(pdf_path)

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
    ]

    md_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=True)
    splits = md_header_splitter.split_text(md_text)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=700, chunk_overlap=60
    )

    splits = text_splitter.split_documents(splits)

    for i in splits:
        metadata = {'Source_path': pdf_path}
        i.metadata = {**i.metadata, **metadata}
    return splits


def make_db(df):
    documents = []
    
    pdf_files = df['Source_path'].unique()
    for pdf_file in tqdm(pdf_files):
        # 문서 로드
        documents.extend(process_pdf(pdf_file))
        
    print(f"Total number of documents: {len(documents)}")

    faiss = FAISS.from_documents(documents, embedding=get_embedding())
    return faiss

In [4]:
def fewshot_db(df):
    df = df.drop('SAMPLE_ID', axis=1)
    df = df.drop('Source_path', axis=1)
    df = df.to_dict(orient='records')
    print("Loaded Fewshot Set:", len(df))
    to_vectorize = ["\n\n".join(normalize_string(value) for value in example.values()) for example in df]
    faiss = FAISS.from_texts(to_vectorize, embedding=get_embedding())
    # bm = BM25Retriever.from_texts(to_vectorize)
    # knn = KNNRetriever.from_texts(to_vectorize, embeddings=get_embedding())
    return faiss

In [5]:
train_df = pd.read_csv('train.csv', encoding='utf-8')
test_df = pd.read_csv('test.csv', encoding='utf-8')

In [6]:
def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for i, doc in enumerate(docs):
        #context += f"Document {i+1}\n"
        doc.page_content = doc.page_content.replace("{", "(")
        doc.page_content = doc.page_content.replace("}", ")")
        
        context += doc.page_content
        context += '\n\n'
    return context.strip()

In [7]:
def setup_llm_pipeline(model_id="meta-llama/Meta-Llama-3.1-8B-Instruct"):
    # 양자화 설정 적용
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,  # 기본적으로 4비트로 로드
        bnb_4bit_use_double_quant=True,  # 두 번 양자화 적용
        bnb_4bit_quant_type="nf4",  # 4비트 양자화 유형 선택
        bnb_4bit_compute_dtype=torch.bfloat16,  # 연산은 bf16으로 수행
        load_in_low_bit = True
        
    )

    # 모델 로드
    model = AutoModelForCausalLM.from_pretrained(
        model_id, 
        quantization_config=bnb_config,
        low_cpu_mem_usage=True
    )

    # 일부 중요한 레이어는 FP16으로 유지
    for name, module in model.named_modules():
        if "attention" in name or "ffn" in name:  # 중요한 레이어 식별 (예: attention 및 ffn)
            module.to(torch.float16)  # 이 부분은 16비트로 유지

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        return_full_text=False,
        max_new_tokens=1024,
        eos_token_id = terminators,
        pad_token_id = tokenizer.eos_token_id
    )

    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return llm
# ghost-x/ghost-8b-beta-1608
# OpenBuddy/openbuddy-llama3.1-8b-v22.3-131k
llm = setup_llm_pipeline()

Loading checkpoint shards: 100%|██████████| 4/4 [00:30<00:00,  7.57s/it]


### 점수

In [8]:
from collections import Counter
def calculate_f1_score(true_sentence, predicted_sentence, sum_mode=True):

    #공백 제거
    true_sentence = ''.join(true_sentence.split())
    predicted_sentence = ''.join(predicted_sentence.split())
    
    true_counter = Counter(true_sentence)
    predicted_counter = Counter(predicted_sentence)

    #문자가 등장한 개수도 고려
    if sum_mode:
        true_positive = sum((true_counter & predicted_counter).values())
        predicted_positive = sum(predicted_counter.values())
        actual_positive = sum(true_counter.values())

    #문자 자체가 있는 것에 focus를 맞춤
    else:
        true_positive = len((true_counter & predicted_counter).values())
        predicted_positive = len(predicted_counter.values())
        actual_positive = len(true_counter.values())

    #f1 score 계산
    precision = true_positive / predicted_positive if predicted_positive > 0 else 0
    recall = true_positive / actual_positive if actual_positive > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1_score

def calculate_average_f1_score(true_sentences, predicted_sentences):
    
    total_precision = 0
    total_recall = 0
    total_f1_score = 0
    
    for true_sentence, predicted_sentence in zip(true_sentences, predicted_sentences):
        precision, recall, f1_score = calculate_f1_score(true_sentence, predicted_sentence)
        total_precision += precision
        total_recall += recall
        total_f1_score += f1_score
    
    avg_precision = total_precision / len(true_sentences)
    avg_recall = total_recall / len(true_sentences)
    avg_f1_score = total_f1_score / len(true_sentences)
    
    return {
        'average_precision': avg_precision,
        'average_recall': avg_recall,
        'average_f1_score': avg_f1_score
    }

### RUN

In [9]:

def extract_answer(response):
    # AI: 로 시작하는 줄을 찾아 그 이후의 텍스트만 추출
    lines = response.split('\n')
    for line in lines:
        line = line.replace('**', '')
        if line.startswith('Answer:'):
            return line.replace('Answer:', '').strip()
        if line.startswith('assistant:'):
            return line.replace('assistant:', '').strip()
    return response.strip()  # AI: 를 찾지 못한 경우 전체 응답을 정리해서 반환

def equal_path(contexts, source_path):
    adjusted_docs = []
    for doc in contexts:
        if doc.metadata['Source_path'] == source_path:
            adjusted_docs.append(doc)
    return adjusted_docs

def rerun(question,context,answer,llm,num_repeat):
    full_template = "<|begin_of_text|>"
    full_template += """<|start_header_id|>system<|end_header_id|>당신은 이전 답변을 검증하는 챗봇입니다. 질문과 문맥, 이전 답변을 참고해서 지시사항을 따르세요. 지시사항을 따를 때 서론 없이 출력하세요.<|eot_id|>"""
    full_template += f"""<|start_header_id|>user<|end_header_id|>Question: {question} \n\nContexts: {context} \n\nPrevious Answer: {answer} \n\n"""
    full_template += """{input}<|eot_id|>"""
    full_template += """<|start_header_id|>assistant<|end_header_id|>"""
    
    prompt = PromptTemplate(template=full_template)
    chain = (
    {
        "input": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
    )
    return chain.invoke("핵심 단어들을 바탕으로, 한 문장으로 요약하세요. 만약 한 문장이라면 그대로 출력하세요.")
    
def run(faiss,dataset,llm,k=2,verbose=False):
    results = []
    source_path = dataset.iloc[0]['Source_path']
    docs = faiss.similarity_search(
        query="",  # 유사도 기반이 아닌 메타데이터 필터링만 사용하므로 query는 빈 값으로
        filter={"Source_path": source_path},
        k = 99,
        fetch_k = 20000
        )
    buff_faiss = FAISS.from_documents(docs, embedding=get_embedding())
    faiss_retriever = buff_faiss.as_retriever(search_type="mmr",search_kwargs={"k": k})
    knn_retriever = KNNRetriever.from_documents(docs, embeddings=get_embedding())
    knn_retriever.k = k
    bm_retriever = OktBM25Retriever.from_documents(docs)
    bm_retriever.k = k
    ensemble_retriever = EnsembleRetriever(retrievers=[faiss_retriever, knn_retriever,bm_retriever], weight=[0.4, 0.3, 0.3])
    
    for i, row in (dataset.iterrows()):
        if source_path != row['Source_path']:   
            source_path = row['Source_path']
            docs = faiss.similarity_search(
                query="",  # 유사도 기반이 아닌 메타데이터 필터링만 사용하므로 query는 빈 값으로
                filter={"Source_path": source_path},
                k = 99,
                fetch_k = 20000
                )
            buff_faiss = FAISS.from_documents(docs, embedding=get_embedding())
            faiss_retriever = buff_faiss.as_retriever(search_type="mmr",search_kwargs={"k": k})
            knn_retriever = KNNRetriever.from_documents(docs, embeddings=get_embedding())
            knn_retriever.k = k
            bm_retriever = OktBM25Retriever.from_documents(docs)
            bm_retriever.k = k
            ensemble_retriever = EnsembleRetriever(retrievers=[faiss_retriever, knn_retriever,bm_retriever], weight=[0.4, 0.3, 0.3])
            
        full_template = "<|begin_of_text|>"
        full_template += """<|start_header_id|>system<|end_header_id|>
당신은 유용한 금융 정보 QnA 챗봇입니다.
질문을 정확히 이해하고, 모든 질문에 답하세요.
답변 시 반드시 문맥 정보를 활용해야합니다. 
서론 없이 핵심 내용을 한 문장으로 작성해주세요. 
<|eot_id|>
"""
        question = row['Question']          
        # full_template += """ """
        contexts = ensemble_retriever.invoke(normalize_string(question))
        # contexts = equal_path(contexts,row['Source_path'])
        contexts = format_docs(contexts)
        full_template += """<|start_header_id|>user<|end_header_id|>Question: {input}\n\n"""
        full_template += f"""Contexts: {contexts}<|eot_id|>"""
        full_template += """<|start_header_id|>assistant<|end_header_id>"""
        
        prompt = PromptTemplate(template=full_template, input_variables=["input"])
        qa_chain = (
        {
            "input": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
        )

        answer = qa_chain.invoke(input=question)
        answer = extract_answer(answer)
        lines = answer.split('\n')
        if  len(lines) > 1:
            previous = answer
            try:
                before = calculate_f1_score(row['Answer'],answer)[2]
            except:
                before = None
            answer = rerun(question=question,
                           context=contexts,
                           answer=answer,
                           llm=llm,
                           num_repeat=1)
        answer = extract_answer(answer)
        results.append({
            "Question": question,
            "Answer": answer,
            "Source": row['Source']
        })
        if verbose:
            print(f"{i}/{len(dataset)}")
            print("Question: ", question, end=" | ")
            print("Context Number |",len(contexts))
            try:
                print(calculate_f1_score(row['Answer'],answer)[2],end=" | ")
            except:
                pass
            print("Answer: ", results[-1]['Answer'])
            try:
                print("Before: ",before," | ",previous)  
                
                previous = None
                before = None
            except:
                pass
            
            try:
                print("REAL Answer: ",row['Answer'])
            except:
                pass
            
            print()
            
        torch.cuda.empty_cache()

    return results

### 케이폴드

In [10]:
# from sklearn.model_selection import KFold
# import copy

# # weight = [0.3,0.3,0.4]
# # train_faiss_db,knn_retriever ,train_bm_retrievier = make_db(train_df) 

# # train_k = 3
# # train_bm_retrievier.k = train_k
# # knn_retriever.k = train_k
# # train_faiss_retriever = train_faiss_db.as_retriever(search_type="mmr",search_kwargs={'k':train_k} )
# # train_ensemble_retriever = EnsembleRetriever(
# #     retrievers=[train_bm_retrievier, knn_retriever,train_faiss_retriever], weights=weight , search_kwargs={'k':train_k}
# # )


# # fewshot_k = 3

# k_folds = 4
# fold_results = []
# kf = KFold(n_splits=k_folds, shuffle=True, random_state=52)
# for fold, (train_index, val_index) in enumerate(kf.split(train_df)):
#     fold_result = []
#     train_set = train_df.iloc[train_index]
#     val_set = train_df.iloc[val_index]

#     faiss = make_db(val_set)

#     pred = run(faiss,val_set, llm, verbose=True)
#     result = pd.DataFrame()
#     result['pred'] = [result['Answer'] for result in pred]
#     val_set.index = range(len(val_set))
#     result['gt'] = val_set['Answer']
        
#     result = calculate_average_f1_score(result['gt'], result['pred'])
#     print(result)
#     fold_results.append(result)
#     break

### 실전

In [11]:
from save_module import save


weight = [0.5,0.5]
test_faiss_db = make_db(test_df)

test_k = 3
# test_bm_retrievier.k = test_k
# #test_knn_retriever.k = test_k
# test_faiss_retriever = test_faiss_db.as_retriever(search_type="mmr",search_kwargs={'k':test_k} )
# test_ensemble_retriever = EnsembleRetriever(
#     retrievers=[test_bm_retrievier, test_faiss_retriever], weights=weight
# )


results = run(test_faiss_db, test_df, llm, verbose=True)
save(results)

  0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 9/9 [00:12<00:00,  1.35s/it]


Total number of documents: 69


c:\ProgramData\anaconda3\envs\llm\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\ProgramData\anaconda3\envs\llm\Lib\site-packages\transformers\models\llama\modeling_llama.py:653: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


0/98
Question:  2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요? | Context Number | 2853
Answer:  2022년 혁신창업사업화자금(융자)의 예산은 2,007,800백만원입니다.

1/98
Question:  중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요? | Context Number | 2853
Answer:  중소벤처기업부의 혁신창업사업화자금(융자) 사업의 목적은 기술력과 사업성이 우수하고 미래 성장가능성이 높은 중소벤처기업의 창업을 활성화하고 고용창출을 도모하고, 중소기업이 보유한 우수 기술의 개발 및 제품화·사업화를 촉진하여 기술기반 중소기업을 육성하는 것입니다.

2/98
Question:  중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요? | Context Number | 2853
Answer:  중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 중소기업진흥에 관한 법률 제66조, 제67조, 제74조입니다.

3/98
Question:  2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요? | Context Number | 2256
Answer:  2010년 신규 지원된 혁신창업사업화자금은 재창업자금(실패 경영인에 대한 재기 지원)을 신규 지원한 것입니다.

4/98
Question:  혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요? | Context Number | 2256
Answer:  2020년 신규 지원된 혁신창업사업화자금은 2,300,000 만원입니다.

5/98
Question:  재창업자금이 재도약지원자금으로 이관된 연도는 언제인가요? | Context Number | 2256
Answer:  재창업자금이 재도약지원자금으로 이관된 연도는 2015년입니다.

6/98
Question:  창업기반지원과 신청 대상이 중복인 자금이 어떤 것이며, 이 자금이 폐지된 연도는 언제인가요? | Context Number | 2853
An

...

51/98
Question:  수직적 재정조정제도와 수평적 재정조정제도의 차이는 무엇인가요? | Context Number | 11248
Answer:  수직적 재정조정제도의 경우, 중앙정부가 지방자치단체에 재원을 이전하는 재정조정의 역할을 함으로써 재정력 격차를 완화하고 재정불균형을 해소하는 기능을 수행합니다. 반면 수평적 재정조정제도의 경우, 지방자치단체 간에 재원을 재배분하는 재정조정의 역할을 함으로써 재정력 격차를 완화하고 재정불균형을 해소하는 기능을 수행합니다.
Before:  None  |  None

52/98
Question:  지방재정조정제도는 어떤 목적을 가지고 있나요? | Context Number | 11449
Answer:  지방교부세와 국고보조금은 수직적 재정조정 기능을 가지는 재원으로 재정불균형을 해소하고 재정분권을 분담하는 중요한 재원으로 작용합니다.
Before:  None  |  지방교부세와 국고보조금은 수직적 재정조정 기능을 가지는 재원으로 재정불균형을 해소하고 재정분권을 분담하는 중요한 재원으로 작용

-   지방교부세는 지방자치단체의 재정부족액을 보전하고, 국고보조금은 재정불균형을 완화하는 데 목적을 두고

-   재정분권의 분담을 통해 지방자치단체의 기본적인 수요에 대한 지출을 촉진하고 재정불균형을 해소하는 데 중요한

기능을 수행

-   지방교부세와 국고보조금은 국가 재정에서 차지하는 지방재정의 비중이 증가하고 있는 만큼, 지방재정조정의

중요성과 지방자치단체의 재정분권 분담의 중요성이 더욱 부각

-   보통교부세는 지방교부세 중 가장 큰 규모로 2022년 기준 55조원으로 전체 지방교부세의 84.7%를 차지

-   보통교부세의 법정률은 2006년부터 내국세의 19.24% 수준으로 유지되고 있으며, 재정부족액을 보전하기

위해 매년 내국세의 신장률만큼 꾸준히 증가하는 추세

-   국고보조금은 2022년 중앙정부 본예산 607.7조원 중 102.3조원으로 재정총지출의 16.8%를 차지

-   사회복지 분야가 55

c:\ProgramData\anaconda3\envs\llm\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


55/98
Question:  재정사업 성과관리제도의 필요성이 대두된 시기와 한국의 재정사업 성과관리제도가 어떤 법에 근거하여 운영되고 있는지 설명하시오. | Context Number | 6928
Answer:  재정사업 성과관리제도의 필요성이 대두된 시기는 2006년 4대 재정개혁을 통해 재정사업 성과관리제도가 정착되기 시작한 시기입니다.
Before:  None  |  핵심재정사업 성과관리제도의 필요성이 대두된 시기는 2006년 4대 재정개혁을 통해 재정사업 성과관리제도가 정착되기 시작한 시기입니다. 이 시기, 정부는 국가재정법을 통해 재정사업 성과관리의 기본 계획과 추진 계획을 수립하는 것을 의무화하였고, 성과관리의 용어, 양식, 절차, 평가 항목 등을 표준화하여 제도 전반에 대한 이해도를 제고했습니다. 또한, 정부는 행정 부담을 경감하면서 재정운용의 책임성과 투명성을 제고하는 방향으로 제도 개편을 진행했습니다.

핵심재정사업 성과관리제도는 국정과제와 연계된 핵심재정사업을 선별하여 성과창출을 위한 관리체계를 신설하였고, 향후 5년간의 국정운영 성공을 뒷받침하는 데 목표를 두었습니다. 이 제도는 부처별 대표 성과지표 위주의 성과공개를 강화하여 국민 관심과 부처단위 성과를 제고하였고, 사업별 성과정보 DB 구축 및 포털 개편을 통해 재정성과정보를 체계적으로 관리하고 공개하는 것을 목표로 하였습니다.

56/98
Question:  청년일자리도약장려금은 어떤 대상을 지원하며, 어떤 방식으로 지원되는가? | Context Number | 12644
Answer:  핵심재정사업 성과관리 제도는 재정사업 성과관리를 위한 제도로, 12대 핵심재정사업(군) 중에서 3대 분야, 12대 핵심재정사업(군)을 선정하여 5년간 관리하여 2027년까지 5년간의 국정운영을 뒷받침하는 동력으로서 제도 안착을 위해 성공적으로 운영되어야 한다.
Before:  None  |  핵심재정사업 성과관리 제도는 재정사업 성과관리를 위한 제도이다. 핵심재정사업 성과관리 제도는 2023년부터 시

c:\ProgramData\anaconda3\envs\llm\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


57/98
Question:  재정성과관리제도는 어떤 측면에서 국정운영과 연결되는가? | Context Number | 6801
Answer:  재정성과관리제도는 재정의 프로그램 단위로 구분하고, 사전 목표와 사후 평가 결과를 중심으로 관리하는 것을 의미합니다. 재정성과관리제도는 재정수지, 국가채무 등 재정 전체에 대한 총량 관리에서는 보지 못하는 미시적인 부분까지 재정을 관리하는 것으로, 총량 관리와 보완적인 관계입니다. 재정성과관리제도는 전략 목표와 우선순위를 중심으로 재정사업을 재구조화하여 국정운영과 연결되며, 지출 우선순위 측면에서 재정을 중장기 시계로 확장시킵니다.
Before:  None  |  None

